In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from airfoil.airfoil import Airfoil, Hole, WingSegment, Decomposer
from numpy.typing import ArrayLike
from itertools import pairwise

import pyvista as pv

In [ ]:
def create_spitfire_wing(
    section_positions:ArrayLike,
    target_real_wingspan = 1000,
):
    
    def ellipse_quadrant(rx, ry, num_points=100):
        # Semi-major and semi-minor axes
        a = rx
        b = ry
        theta = np.linspace(0, np.pi/2, num_points)
        # Parametric equations for ellipse
        x = a * np.cos(theta)
        y = b * np.sin(theta)
        
        return np.array([x, y]).transpose()

    target_half_span     = target_real_wingspan/2

    root_chord_centerline        = 100.0 / 222.5 * target_half_span
    half_span                    = 222.5 / 222.5 * target_half_span
    front_spar_from_leading_edge =  24.5 / 222.5 * target_half_span
    wing_axis_from_leading_edge  =  35.5 / 222.5 * target_half_span
    dihedral_deg                 = 4
    washout_deg = 2

    leading_edge  = ellipse_quadrant(half_span, wing_axis_from_leading_edge, num_points=50)

    def local_chord_length(span_x:float):
        return root_chord_centerline*np.sqrt(1-4*(span_x/(half_span*2))**2)

    def local_chord_setback(span_x:float):
        return np.interp(span_x,*leading_edge[::-1].transpose())

    def local_thickness(span_x):
        # Should be changed a bit;
        # 13% to 9% where the outboard measurement of aurfoil thickness is taken at 160 from root
        return (0.13-0.09)*span_x/half_span+0.09

    def local_airfoil(span_x):
        span_fraction = span_x/half_span
        rotation_center = -local_chord_length(span_x)*0.2
        wing_slope = np.tan(np.deg2rad(dihedral_deg))
        af = (
            Airfoil.from_naca4(
                max_camber          = 0.02,
                max_camber_position = 0.2,
                max_thickness       = local_thickness(span_x),
                chord_length        = local_chord_length(span_x),
            )
            .with_translation((-rotation_center, 0))
            .with_rotation(-span_fraction*washout_deg)
            .with_translation((rotation_center, 0))
            .with_translation(
                (
                    wing_axis_from_leading_edge-local_chord_setback(span_x),
                    wing_slope*span_x
                )
            )
        )
        return af
    
    results = []
    for section_position in section_positions:
        results.append(local_airfoil(section_position))
    return results


In [ ]:
import matplotlib.ticker as plticker

section_positions=np.array([
    0,
    100,
    200,
    300,
    400,
    450,
])
sections = create_spitfire_wing(
    section_positions    = section_positions,
    target_real_wingspan = 1000,
)
fig,ax = plt.subplots(figsize=(20,5))
for af in sections:
    af.plot_raw(ax)
ax.xaxis.set_major_locator(plticker.MultipleLocator(10.0))
ax.yaxis.set_major_locator(plticker.MultipleLocator( 5.0))
ax.grid()
#ax.set_aspect("equal")

In [ ]:
chunks_a, chunks_b = [*pairwise(sections)], [*reversed([*pairwise(reversed(sections))])]

def add_holes(afs:tuple[Airfoil,Airfoil])->tuple[Airfoil,Airfoil]:
    holes = [
        Hole(diameter_mm=3, position=np.array([ 50,8])),
        Hole(diameter_mm=3, position=np.array([100,8])),
    ]
    a, b = afs
    a = a.with_holes(holes)
    b = b.with_holes(holes)
    return a, b

chunks_a[0] = add_holes(chunks_a[0])
chunks_b[0] = add_holes(chunks_b[0])

airfoil_pairs_a:list[WingSegment] = [
    WingSegment(a, b, length=length)
    for (a, b), length
    in zip(
        chunks_a,
        np.diff(section_positions)
    )
]
airfoil_pairs_b:list[WingSegment] = [
    WingSegment(a, b, length=length)
    for (a, b), length
    in zip(
        chunks_b,
        np.diff(section_positions)
    )
]

In [ ]:
import pyvista as pv
pt = pv.Plotter()
offset = 30
for chunk in airfoil_pairs_a:
    offset += chunk.length/2
    m = chunk.to_mesh()
    pt.add_mesh(m.translate((offset,0,0)), opacity=0.5)
    offset += chunk.length/2
offset = -30
for chunk in airfoil_pairs_b:
    offset -= chunk.length/2
    m = chunk.to_mesh()
    pt.add_mesh(m.translate((offset,0,0)), opacity=0.5)
    offset -= chunk.length/2
pt.show()

In [ ]:
from itertools import chain, repeat, count
foam_cut_list = []
for side, wing_segment in chain(
        zip(
            repeat("A"),
            airfoil_pairs_a
        ),
        zip(
            repeat("B"),
            airfoil_pairs_b
        )
    ):
    bounds = wing_segment.to_mesh().bounds
    foam_cut_list.append({
        "side"            : side,
        "foam_width_mm"  : bounds.x_max-bounds.x_min,
        "foam_depth_mm": np.ceil((bounds.y_max-bounds.y_min + 10)/10)*10,
        "foam_height_mm"    : 30 if (bounds.z_max-bounds.z_min) < 30 else 50,
        "wing_segment"    : wing_segment,
    })
segments = pd.DataFrame(foam_cut_list)
segments

In [ ]:
from util.pyvista_mock import axis
from util.path_planning import (
    blur1d,
    map_to_range,
    project_line_to_plane,
    geometric_curvature2,
    geometric_curvature,
    remove_sequential_duplicates,
    ensure_closed,
)
from util.pyvista_helpers import create_ruled_surface
from dataclasses import dataclass, replace


@dataclass
class MachineSetup:
    wing_segment:WingSegment
    foam_width :float
    foam_depth :float
    foam_height:float = 30
    plane_spacing:float = 227
    
    def with_recentered_part(self):
        foam_center = np.array([
            0,
            self.foam_depth/2,
            self.foam_height/2,
        ])
        offset = foam_center - self.wing_segment.bounding_center()
        return replace(
            self,
            wing_segment = self.wing_segment.with_translation(offset[-2:])
        )

    def plot(self, state:tuple[float,float,float,float]|ArrayLike):
        _state = np.array(state)
        mesh_foam = pv.Box((
            -self.foam_width/2,self.foam_width/2,
            0,self.foam_depth,
            0,self.foam_height
        ))
        state_a, state_b = self.state_to_line(*_state)
        mesh_state = pv.Line(state_a,state_b)
        decomposer = Decomposer()
        mesh_target = self.wing_segment.to_mesh(decomposer)

        a,b, speed = self.prepare_instructions(decomposer)

        cut_surface = create_ruled_surface(a,b)
        cut_surface.cell_data["speed mm/s"] = speed[:-1]

        pt = pv.Plotter()
        pt.add_mesh(pv.MultipleLines(a),"green")
        pt.add_mesh(pv.MultipleLines(b),"green")
        pt.add_mesh(
            mesh_foam.extract_all_edges(),
            color="teal",
            line_width=2,
        )
        pt.add_mesh(cut_surface,scalars="speed mm/s", cmap="viridis")
        pt.add_mesh(mesh_state, color="red", line_width=2)
        if mesh_target:
            pt.add_mesh(mesh_target)
        pt.add_mesh(axis(
            (-self.plane_spacing/2, *_state[:2]), side="L"
        ), color="white", opacity=0.1)
        pt.add_mesh(axis(
            ( self.plane_spacing/2, *_state[2:]), side="R"
        ), color="white", opacity=0.1)
        pt.camera_position = (
            (-self.foam_width*1.1,-self.foam_width*3,self.foam_height*3),
            (-self.foam_width*0.3,0,0),
            (0,0,1)
        )
        pt.enable_parallel_projection()
        pt.show()


    def state_to_line(self, x:float, y:float, z:float, a:float):
        return (
            (-self.plane_spacing/2,x,y),
            ( self.plane_spacing/2,z,a),
        )

    def states_to_curves(self,states):
        x,y,z,a=states.T
        return (
            np.vstack((np.ones_like(x) * -self.plane_spacing/2, x,y)).T,
            np.vstack((np.ones_like(x) *  self.plane_spacing/2, z,a)).T,
        )
    
    def prepare_instructions(self,decomposer:Decomposer):
        a, b = self.wing_segment.decompose(decomposer)
        a = ensure_closed(remove_sequential_duplicates(np.concat(a)))
        b = ensure_closed(remove_sequential_duplicates(np.concat(b)))
        a_3d = np.insert(a, 0, -self.wing_segment.length/2, axis=-1)
        b_3d = np.insert(b, 0,  self.wing_segment.length/2, axis=-1)
        afa_projected = []
        afb_projected = []
        for a_3di, b_3di in zip(
            a_3d,
            b_3d,
        ):
            afa_projected.append(project_line_to_plane(a_3di, b_3di, "yz", -self.plane_spacing/2))
            afb_projected.append(project_line_to_plane(a_3di, b_3di, "yz",  self.plane_spacing/2))
        afa_projected = np.array(afa_projected)
        afb_projected = np.array(afb_projected)
        speed = map_to_range(
            blur1d(np.max([
                geometric_curvature(a),
                geometric_curvature(b),
            ],axis=0)),
            300,
            200
        )
        speed *= (
             (np.linalg.norm(np.diff(afa_projected))/np.linalg.norm(np.diff(a)))
            +(np.linalg.norm(np.diff(afb_projected))/np.linalg.norm(np.diff(b)))
        )/2
        return afa_projected, afb_projected, speed

In [ ]:
seg = 0
ms = MachineSetup(
    wing_segment = segments.loc[seg,"wing_segment"],
    foam_width   = segments.loc[seg,"foam_width_mm"],
    foam_depth   = segments.loc[seg,"foam_depth_mm"],
    foam_height  = segments.loc[seg,"foam_height_mm"],
).with_recentered_part()

state = np.array([0,0,0,0])
ms.plot(state)